In [1]:
import pandas as pd
import datetime, pytz
import vectorbt as vbt
import numpy as np
import matplotlib.pyplot as plt
import json

import talib
import pandas_ta
import time

from importlib import reload


import sys 
sys.path.append('../../modules/')

from forwardtest import ForwardTest
from vbt_indicator import Indicators

c:\Users\slavi\anaconda3\envs\Bruno_proj\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [11]:
help(vbt.talib('WILLR').run)

Help on method run:

run(high, low, close, timeperiod=Default(14), short_name='willr', hide_params=None, hide_default=True, **kwargs) method of vectorbt.indicators.factory.MetaIndicatorBase instance
    Run `WILLR` indicator.
    
    * Inputs: `high`, `low`, `close`
    * Parameters: `timeperiod`
    * Outputs: `real`
    
    Pass a list of parameter names as `hide_params` to hide their column levels.
    Set `hide_default` to False to show the column levels of the parameters with a default value.
    
    Other keyword arguments are passed to `vectorbt.indicators.factory.run_pipeline`.



In [26]:
def bigwill(High, Low, Close, ao1, ao2, stoch_window, willr_period):
    src = (High+ Low) 
    ao = vbt.talib('SMA').run(src, ao1).real.to_numpy() - vbt.talib('SMA').run(src, ao2).real.to_numpy()
    stoch_rsi = vbt.talib('STOCHRSI').run(Close, stoch_window).fastk.to_numpy()
    willr = vbt.talib('WILLR').run(High, Low, Close, willr_period).real.to_numpy()

    entries = (ao > 0) & (willr < -85)
    exits = ((ao < 0) & (stoch_rsi > 0.2)) | (willr > -10)

    return entries, exits

BigWill = vbt.IndicatorFactory(
    class_name= 'BigWill',
    short_name = 'BW',
    input_names = ["High", "Low", "Close"] ,
    param_names = ["ao1", "ao2", "stoch_window", "willr_period"],
    output_names= ["entries", "exits"]
    ).from_apply_func(bigwill)

In [27]:
params = {
    'backtest': {
        'start': '2017-10-1',
        'end': '2018-1-1',
        'length': {
            'weeks': 4  #durée pendant laquelle on optimise les paramètres
        },
        'forward': {
            'weeks': 2  #durée pendant laquelle on joue les paramètres optimisés
        },
        
    },
    'strat':{
        'download_params': {
            'symbols': 'BTCUSDT',
            'interval': '1h',            
        },
        'get_params': {
            'column': ['High', 'Low', 'Close']
        },
        'run_params': { 
            'ao1': {'start': 5, 'stop': 10, 'step': 1},
            'ao2': {'start': 20, 'stop': 30, 'step': 1},
            'stoch_window': {'start': 10, 'stop': 20, 'step': 1},
            'willr_period': {'start': 10, 'stop': 20, 'step': 1},
            'param_product': 'True'
        }, 
    },
    'portfolio': {'freq': 'H' ,
                  'fees' : 0.1/100,
                  'size_type': 'Percent',
                  'size': .1} #proportion du portefeuille à investir dans chaque trade
}



In [28]:
test = ForwardTest(BigWill, params)
test.backtest()

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-10-29 00:00:00+00:00 and 2017-11-12 00:00:00+00:00 finished at 2023-07-17 20:15:22.797830
Cash: 996.1723862958162


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-12-10 00:00:00+00:00 and 2017-12-24 00:00:00+00:00 finished at 2023-07-17 20:18:37.348901
Cash: 1003.6005347337921


0it [00:00, ?it/s]

1003.6005347337921

In [29]:
pf = test.aggregate_results()
pf.stats()


Start                         2017-10-29 00:00:00+00:00
End                           2017-12-23 23:00:00+00:00
Period                                 56 days 00:00:00
Start Value                                       100.0
End Value                                    100.360053
Total Return [%]                               0.360053
Benchmark Return [%]                          133.38779
Max Gross Exposure [%]                        10.429055
Total Fees Paid                                0.311821
Max Drawdown [%]                               1.356308
Max Drawdown Duration                  28 days 20:00:00
Total Trades                                         16
Total Closed Trades                                  15
Total Open Trades                                     1
Open Trade PnL                                -0.010027
Win Rate [%]                                  66.666667
Best Trade [%]                                 5.902148
Worst Trade [%]                               -4